# Hotstart

Many times one needs to split a simulation in intermittent steps. This is true for long hindacts and forecasts. This Notebook outlines the hotstart process using the `schism` solver. 


- Requires:

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.model as pm

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env PATH. This is not needed when we are running from python.

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
import numpy
cpath = numpy.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

In [ ]:
import geopandas as gp
# use cartopy to get coastlines
import cartopy.feature as cf

Without loss of generality we are using the natural earth's intermediate resolution dataset as coastlines

In [ ]:
cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

In [ ]:
ne_i

## Setup

The minimum information required is an initial model and corresponding run. We set this one up. For more info see the [MODEL](./MODEL.ipynb) Notebook.


In [ ]:
#define in a dictionary the properties of the model..
model={'solver_name':'schism',
     'geometry':{'lon_min' : -25,'lon_max' : -12.,'lat_min' : 56.,'lat_max' : 74.}, 
     'coastlines':ne_i,
     'mesh_generator' : 'jigsaw', # set grid generator 
     'meteo_engine':'cfgrib',
     'start_date':'2018-10-1 0:0:0',
     'time_frame':'24H',
     'meteo_source' : './data/uvp_2018100100.grib', #path to meteo files
     'dem_source' : './data/dem.nc', #path to dem file
     'rpath':'./test/hotstart/step1/', #location of calc folder
     'parameters':{'dt':400, 'rnday':1., 'nhot':1, 'ihot':0,'nspool':9, 'ihfskip':36, 'nhot_write':108 }, # set param.nml components
     'scribes':2,
      }

* Note that the parameters arguments will override relevant settings

## Initialize

In [ ]:
#initialize a model
b = pm.set(**model)

In [ ]:
#Execute
b.execute()

*Note* that hotstart files have been created (check ./test/hotstart/step1/outputs)

## Use the cast module to do a hotstart

In [ ]:
from pyposeidon.utils import cast
import pandas as pd
import datetime

In [ ]:
# folder for the new run hotstarting from the step1 folder
new_path = './test/hotstart/step2/'

In [ ]:
# new meteo file
mf = './data/uvp_2018100112.grib'

In [ ]:
b = pyposeidon.model.read("./test/hotstart/step1/schism_model.json")

In [ ]:
rs = cast.set(solver_name='schism',
        model=b, #reference model
        ppath=b.rpath, #original path
        cpath=new_path, #new path
        meteo=mf, #new meteo
        sdate=pd.to_datetime(b.rdate) + pd.DateOffset(hours=12), #new start time
        copy = True # optional, default is simlink for common files      
            ) 



In [ ]:
rs.run(execute=False)

## Configure without the cast class

In [ ]:
# read json from previous folder
c = pyposeidon.model.read("./test/hotstart/step1/schism_model.json")

In [ ]:
sdate = pd.to_datetime(c.start_date) + pd.DateOffset(hours=12)
edate = pd.to_datetime(c.end_date) + pd.DateOffset(hours=12)

In [ ]:
rdate = pd.to_datetime(c.rdate)

In [ ]:
hotout = int((sdate - rdate).total_seconds() / c.params["core"]["dt"])
hotout

In [ ]:
# create the hotstart file
c.hotstart(it=hotout)

In [ ]:
## copy folder removing outputs
! cp -r ./test/hotstart/step1/ ./test/hotstart/step2_
! rm -r ./test/hotstart/step2_/outputs
! rm -r ./test/hotstart/step2_/jigsaw # if it exists
! rm ./test/hotstart/step2_/*.log

In [ ]:
## copy hotstart
! cp ./test/hotstart/step1/outputs/hotstart_it\=108.nc ./test/hotstart/step2_/hotstart.nc

In [ ]:
#update the start_date (needed for meteo)
c.start_date = sdate
c.end_date = edate

In [ ]:
# define new meteo
c.meteo_source = './data/uvp_2018100112.grib'

In [ ]:
#set new path
c.rpath = "./test/hotstart/step2_/"

In [ ]:
# compute meteo
c.force()

In [ ]:
#output to file
c.to_force(
    c.meteo.Dataset,
    vars=["msl", "u10", "v10"],
    rpath = c.rpath,
    date=c.rdate # keep the original reference date for setting time correctly
    )


In [ ]:
#update param.nml
c.start_date = pd.to_datetime(c.rdate) #revert so that the reference date is the same
c.config(
        output=True, 
        config_file='./test/hotstart/step1/param.nml', 
        parameters={'rnday':1.5, 'ihot':2}
        )

In [ ]:
# save json file
c.parameters.update(
    {
    "start_hour": rdate.hour,
    "start_day": rdate.day,
    "start_month": rdate.month,
    "start_year": rdate.year,
            })

c.misc = {}

c.config_file = './test/hotstart/step2_/param.nml'

c.start_date = sdate
c.save()

## check meteo

In [ ]:
import xarray as xr

In [ ]:
m1 = xr.open_dataset('./test/hotstart/step2/sflux/sflux_air_1.0001.nc')

In [ ]:
m2 = xr.open_dataset('./test/hotstart/step2_/sflux/sflux_air_1.0001.nc')

In [ ]:
m1.equals(m2)

In [ ]:
m1.close()
m2.close()

In [ ]:
! diff -rq ./test/hotstart/step2 ./test/hotstart/step2_ 